# Week-3 Notebook-3

​

​
The first few cells are the same as previous notebook so I'm not adding Markdowns here.



In [1]:
import pandas as pd
url = r'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
tables = pd.read_html(url)
tables = tables[0]
tables.columns=tables.iloc[0]
tables = tables.drop(0)

In [2]:
tables = tables.replace("Not assigned","NaN")

In [3]:
tables = tables.set_index("Borough",drop=0)
tables = tables.drop("NaN", axis=0)
tables['Neighbourhood'] = [row[-2] if row[-1]=='NaN' else row[-1] for row in tables.itertuples()]
tables = tables.reset_index(drop=True)

In [4]:
tables= tables.groupby(['Postcode','Borough']).agg(",".join)
tables = tables.reset_index(drop=0)

In [5]:
tables.shape

(103, 3)

In [6]:
data=[]
file = open('Geospatial_Coordinates.csv','r')
for i in file:
    i = str(i).strip()
    i = i.split(',')
    data.append(i)

In [7]:
data = pd.DataFrame(data)
data.columns=data.iloc[0]
data = data.drop(0)

In [8]:
tables = pd.concat([tables.set_index('Postcode'),data.set_index('Postal Code')], axis=1, join='inner').reset_index(drop=0)
tables = tables.rename(columns={'index':'Postcode'})
tables.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge,Malvern",43.8066863,-79.1943534
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union",43.7845351,-79.1604971
2,M1E,Scarborough,"Guildwood,Morningside,West Hill",43.7635726,-79.1887115
3,M1G,Scarborough,Woburn,43.7709921,-79.2169174
4,M1H,Scarborough,Cedarbrae,43.773136,-79.2394761


Taking help from Week 2 Assignment on Foursquare APIs, I'll need to cluster the data

In [9]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')


CLIENT_ID = 'GOB3JCHBYQXPYCANCHMHZFU1JD3JHVQ1JKTDUYKSZQRGG1IH' # your Foursquare ID
CLIENT_SECRET = 'TYXODU3E0FYDVNFBAECEEV2AMDBIK0NYPYRN5MH2FBKNFEVZ' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print("Credentials Approved !")

Libraries imported.
Credentials Approved !


Defining LIMIT of no. of venues, Radius around which to be searched.

Also selecting unique Borough names with 'TORONTO' in it

In [10]:
LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 1500 # define radius
city_list = [s for s in tables.Borough.unique() if 'toronto' in s.lower()]

Similar to Week-2 assignment, defining a function to return all neighbourhoods of Toronto using FourSquare

In [11]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighbourhood Latitude', 
                  'Neighbourhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [12]:
toronto_venues = getNearbyVenues(names=city_list,
                                   latitudes=tables['Latitude'],
                                   longitudes=tables['Longitude']
                                  )

East Toronto
Central Toronto
Downtown Toronto
West Toronto


In [13]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

In [14]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()

Func returns top n venues 

In [15]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    return row_categories_sorted.index.values[0:num_top_venues]

In [16]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

creating model for k-means 

In [17]:
# set number of clusters
kclusters = 4

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

In [18]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = toronto_venues

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

Printing the 4 clusters superimposed in map of Toronto

In [20]:
geolocator = Nominatim(user_agent="toronto_explorer")
location = geolocator.geocode('Toronto')
latitude = location.latitude
longitude = location.longitude

map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Neighbourhood Latitude'], toronto_merged['Neighbourhood Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [float(lat),float(lon)],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

Details of Clusters

In [21]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0,toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Neighbourhood Latitude,Venue Longitude,Venue Category,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,43.7845351,-79.163085,Bar,0,History Museum,Bar,Spa,Rental Car Location,Pizza Place,Mexican Restaurant,Medical Center,Korean Restaurant,Intersection,Fast Food Restaurant
2,43.7845351,-79.162438,History Museum,0,History Museum,Bar,Spa,Rental Car Location,Pizza Place,Mexican Restaurant,Medical Center,Korean Restaurant,Intersection,Fast Food Restaurant


In [22]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1,toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Neighbourhood Latitude,Venue Longitude,Venue Category,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,43.8066863,-79.199056,Fast Food Restaurant,1,Fast Food Restaurant,Spa,Rental Car Location,Pizza Place,Mexican Restaurant,Medical Center,Korean Restaurant,Intersection,History Museum,Electronics Store


In [23]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2,toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Neighbourhood Latitude,Venue Longitude,Venue Category,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
11,43.7709921,-79.221156,Coffee Shop,2,Coffee Shop,Korean Restaurant,Spa,Rental Car Location,Pizza Place,Mexican Restaurant,Medical Center,Intersection,History Museum,Fast Food Restaurant
12,43.7709921,-79.223078,Coffee Shop,2,Coffee Shop,Korean Restaurant,Spa,Rental Car Location,Pizza Place,Mexican Restaurant,Medical Center,Intersection,History Museum,Fast Food Restaurant
13,43.7709921,-79.214502,Korean Restaurant,2,Coffee Shop,Korean Restaurant,Spa,Rental Car Location,Pizza Place,Mexican Restaurant,Medical Center,Intersection,History Museum,Fast Food Restaurant


In [24]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3,toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Neighbourhood Latitude,Venue Longitude,Venue Category,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
3,43.7635726,-79.189914,Pizza Place,3,Spa,Rental Car Location,Pizza Place,Mexican Restaurant,Medical Center,Intersection,Electronics Store,Breakfast Spot,Korean Restaurant,History Museum
4,43.7635726,-79.191537,Electronics Store,3,Spa,Rental Car Location,Pizza Place,Mexican Restaurant,Medical Center,Intersection,Electronics Store,Breakfast Spot,Korean Restaurant,History Museum
5,43.7635726,-79.191000,Spa,3,Spa,Rental Car Location,Pizza Place,Mexican Restaurant,Medical Center,Intersection,Electronics Store,Breakfast Spot,Korean Restaurant,History Museum
6,43.7635726,-79.190720,Mexican Restaurant,3,Spa,Rental Car Location,Pizza Place,Mexican Restaurant,Medical Center,Intersection,Electronics Store,Breakfast Spot,Korean Restaurant,History Museum
7,43.7635726,-79.193406,Rental Car Location,3,Spa,Rental Car Location,Pizza Place,Mexican Restaurant,Medical Center,Intersection,Electronics Store,Breakfast Spot,Korean Restaurant,History Museum
8,43.7635726,-79.192286,Medical Center,3,Spa,Rental Car Location,Pizza Place,Mexican Restaurant,Medical Center,Intersection,Electronics Store,Breakfast Spot,Korean Restaurant,History Museum
9,43.7635726,-79.189490,Intersection,3,Spa,Rental Car Location,Pizza Place,Mexican Restaurant,Medical Center,Intersection,Electronics Store,Breakfast Spot,Korean Restaurant,History Museum
10,43.7635726,-79.190466,Breakfast Spot,3,Spa,Rental Car Location,Pizza Place,Mexican Restaurant,Medical Center,Intersection,Electronics Store,Breakfast Spot,Korean Restaurant,History Museum
